### 1) Getting dowloads statistics table schema

In [2]:
from dotenv import load_dotenv
from pyarrow import total_allocated_bytes

from src.api.pypi import PyPIBigQuery
from src.db.bigquery.utils import get_table_schema

load_dotenv()

src = PyPIBigQuery().sources['download_statistics']
res = get_table_schema(src['project_id'], src['dataset_id'], src['table_id'])

display(res)

/home/evren/miniconda3/envs/ScaletechTask/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,column_name,data_type,is_nullable
0,timestamp,TIMESTAMP,NO
1,country_code,STRING,YES
2,url,STRING,NO
3,project,STRING,NO
4,file,"STRUCT<filename STRING, project STRING, versio...",NO
5,details,"STRUCT<installer STRUCT<name STRING, version S...",YES
6,tls_protocol,STRING,YES
7,tls_cipher,STRING,YES


In [4]:
res.loc[res['column_name'] == 'file', 'data_type'].values[0]
# res.loc[res['column_name'] == 'details', 'data_type'].values[0]

'STRUCT<filename STRING, project STRING, version STRING, type STRING>'

### 2) Cost of Queries (Grouping: Date)b

In [10]:
from src.db.bigquery.utils import get_query_size_and_cost, print_query_size_and_cost

packages = [
    'apache-airflow',
    'dbt',
    'pyspark',
    'pandas',
    'sqlalchemy',
    'great-expectations',
    'prefect',
    'kafka-python',
    'snowflake-connector-python',
    'duckdb',
    'google-cloud-bigquery'
]

wrp = PyPIBigQuery()
table_ref = wrp.get_table_ref('download_statistics')

res = {}
for pkg in packages:

    q = \
        f"""
        SELECT
          DATE(timestamp) AS date,
          COUNT(*) AS download_count
        FROM `{table_ref}`
        WHERE file.project = "{pkg}"
        GROUP BY date
        ORDER BY date DESC
        ;
        """

    pkg_res = get_query_size_and_cost(q, size_as='GB')

    print(pkg)
    print_query_size_and_cost(pkg_res)
    print()

    res.update({pkg: pkg_res})

gbs = 0
for pkg, stats in res.items():
    gbs += stats['n_processed']

print(f'Separately, total GB processed is {gbs} GB')
print('If queried all at once:')

packages_str = ', '.join([f'"{p}"' for p in packages])

q = \
    f"""
    SELECT
      DATE(timestamp) AS date,
      COUNT(*) AS download_count
    FROM `{table_ref}`
    WHERE file.project IN ({packages_str})
    GROUP BY date
    ORDER BY date DESC
    ;
    """

total_res = get_query_size_and_cost(q, size_as='GB')
print_query_size_and_cost(total_res)

apache-airflow
--- Query Dry Run Results ---
Total GB Processed: 66.828091683 bytes
Total GB Billed: 0.0 bytes
Estimated Cost: $0.00 USD

dbt
--- Query Dry Run Results ---
Total GB Processed: 49.608331636 bytes
Total GB Billed: 0.0 bytes
Estimated Cost: $0.00 USD

pyspark
--- Query Dry Run Results ---
Total GB Processed: 56.283994797 bytes
Total GB Billed: 0.0 bytes
Estimated Cost: $0.00 USD

pandas
--- Query Dry Run Results ---
Total GB Processed: 178.440383438 bytes
Total GB Billed: 0.0 bytes
Estimated Cost: $0.00 USD

sqlalchemy
--- Query Dry Run Results ---
Total GB Processed: 138.872429778 bytes
Total GB Billed: 0.0 bytes
Estimated Cost: $0.00 USD

great-expectations
--- Query Dry Run Results ---
Total GB Processed: 50.825996703 bytes
Total GB Billed: 0.0 bytes
Estimated Cost: $0.00 USD

prefect
--- Query Dry Run Results ---
Total GB Processed: 52.308909578 bytes
Total GB Billed: 0.0 bytes
Estimated Cost: $0.00 USD

kafka-python
--- Query Dry Run Results ---
Total GB Processed: 53

### 3) Cost of Queries (Grouping: Date/Version/Country)

In [14]:
from src.db.bigquery.utils import get_query_size_and_cost, print_query_size_and_cost

packages = [
    'apache-airflow',
    'dbt',
    'pyspark',
    'pandas',
    'sqlalchemy',
    'great-expectations',
    'prefect',
    'kafka-python',
    'snowflake-connector-python',
    'duckdb',
    'google-cloud-bigquery'
]

wrp = PyPIBigQuery()
table_ref = wrp.get_table_ref('download_statistics')

res = {}
for pkg in packages:

    q = \
        f"""
        SELECT
          DATE(timestamp) AS date,
          file.version AS version,
          country_code,
          COUNT(*) AS download_count
        FROM `{table_ref}`
        WHERE file.project = "{pkg}"
        GROUP BY date, version, country_code
        ORDER BY date DESC, version DESC
        ;
        """

    pkg_res = get_query_size_and_cost(q, size_as='GB')

    print(pkg)
    print_query_size_and_cost(pkg_res)
    print()

    res.update({pkg: pkg_res})

gbs = 0
for pkg, stats in res.items():
    gbs += stats['n_processed']

print(f'Separately, total GB processed is {gbs} GB')
print('If queried all at once:')

packages_str = ', '.join([f'"{p}"' for p in packages])

q = \
    f"""
    SELECT
      DATE(timestamp) AS date,
      file.version AS version,
      country_code,
      COUNT(*) AS download_count
    FROM `{table_ref}`
    WHERE file.project IN ({packages_str})
    GROUP BY date, version, country_code
    ORDER BY date DESC, version DESC
    ;
    """

total_res = get_query_size_and_cost(q, size_as='GB')
print_query_size_and_cost(total_res)

apache-airflow
--- Query Dry Run Results ---
Total GB Processed: 98.534558678 bytes
Total GB Billed: 0.0 bytes
Estimated Cost: $0.00 USD

dbt
--- Query Dry Run Results ---
Total GB Processed: 79.124276545 bytes
Total GB Billed: 0.0 bytes
Estimated Cost: $0.00 USD

pyspark
--- Query Dry Run Results ---
Total GB Processed: 91.966336978 bytes
Total GB Billed: 0.0 bytes
Estimated Cost: $0.00 USD

pandas
--- Query Dry Run Results ---
Total GB Processed: 299.859221956 bytes
Total GB Billed: 0.0 bytes
Estimated Cost: $0.00 USD

sqlalchemy
--- Query Dry Run Results ---
Total GB Processed: 220.72555746 bytes
Total GB Billed: 0.0 bytes
Estimated Cost: $0.00 USD

great-expectations
--- Query Dry Run Results ---
Total GB Processed: 77.39835508 bytes
Total GB Billed: 0.0 bytes
Estimated Cost: $0.00 USD

prefect
--- Query Dry Run Results ---
Total GB Processed: 84.226207228 bytes
Total GB Billed: 0.0 bytes
Estimated Cost: $0.00 USD

kafka-python
--- Query Dry Run Results ---
Total GB Processed: 83.8